# 1. Max Capacity between two locations

## Problem breakdown

- Given a list of tunnels in the form of `[(start_location, end_location, max cars/h) ...]`
- Given the source and destination
- Calculate the theoretical maximum capcacity between the source and destination
- Can ignore the cars (flows) that exit the system or enter the system anywhere else

## Modelling

The observation is that this is essentially a graph problem, where a Directed Graph has $N$ nodes
and $M$ edges. An edge represent a tunnel $A \to B$ in the system, and a node is either the
start location or the end location of a tunnel. 

Each edge $E$ carries a positive number representing the maximum number of cars passing through this edge, i.e. the maximum flow.

Collectively, the edge flows limit the total flow passing from $Source$ to $Desination$.


## Assumption

I will assume the following constraints:

- the initial edge flows can only be positive (as it doesn't make sense to have `-10` cars passing through a tunnel)
- every car that enters the system at the start location will only exit at the destination location, i.e. they won't exit earlier via any side route
- there is at least one path from the source to destination

## The Algorithm and the pseudo-code

I will base my solution on Edmonds-Karp algorithm.

Firstly, I need to build up a residual graph from the given list of edges (tunnels).

The residual graph is a Directed Graph where each edge $E$ in the original input has a counterpart back-edge $\overleftarrow{E}$ with 0 flow. Namely if there is an edge $E$ connecting $A \to B$, then its backedge $\overleftarrow{E}$ connects $B \to A$

The back-edge serves 2 purposes:

- it records how much existing edge capacity has been utilized
- it lets the algorithm to continue from a previously evaluated path to discover new paths, e.g
$\begin{cases} A \to B \\
A \to B \to C \\
A \to B \to C \to D \dots 
\end{cases}
$

The main procedure works by iteratively picking one path from $Source$ to $Destination$
and finding out its capacity $E_c$, which is the lowest capacity among all the edges in this path. This is based on the constraint theory: the edge with the lowest capacity limits the total capacity of the path.

The algorithm will saturate this edge, setting its flow to 0 and lower the capcity of all the other edges in this path by the same amount $E_c$.

It then finds another non-saturated path and repeat the same process.

The algorithm terminates when it can no longer find a path that is not saturated, i.e. all the paths that connect the $Source$ to $Destination$ contain at least one edge with `0` flow. This is the moment the system runs with the maximum capacity, which is the sum of the all previously discovered capacities, $\sum E_c$


### Pseudo-code in Python

```python
def max_flow(edges, src, sink):
    residual_graph = create_residual_graph(edges)
    result = 0
    while True:
        path = extract_path(residual_graph, src, sink)
        if not path:
            break
        min_flow = min_capacity(path)
        result += min_flow
        for (fr, to_, weight) in path:
            update_residual_graph(residual_graph, fr, to_, -min_flow)
            update_residual_graph(residual_graph, to_, fr, min_flow)
    return result
```

- "destination" is called "sink" in this pseudo-code to follow the converion established by Edmonds-Karp.
- `create_residual_graph()` builds up a graph in the adjacency list representation using Python's `defaultdict`.
- `extract_path()` performs a breath-first search to discover an unsatured path
- `update_residual_graph()` modifies the residual graph, taking contribution from each edge in the current path 

## Complexity

### Runtime

The runtime complexity of this algorithm is $O(N \cdot M^2)$ (recall that $N$ is the number of nodes and $M$ the number of edges). Here is the reasoning:

- the number of iterations is bounded by the number of nodes, $N$
- each iteration takes at maximum $O(M)$ to find an unsaturated path
- for each path, it takes up to $O(M)$ to update the flow values (taking contributions and marking certain edge saturated)

hence $O(N \cdot M \cdot M) = O(N \cdot M^2)$

The time taken to create the residual graph is bounded at $O(N + M)$ therefore it does not contribute to the overall runtime complexity (as it is dominated by the max flow computation)

### Space

The space complexity varies, depending on the graph representation.

Here I use the adjacency list representation (based on Python's dictionary, which is a hash map) therefore it takes $O(N \cdot M)$ space (plus some additional bookkeeping space needed by the garbage collector)

## Working demo

A working demo of this solution can be found at `nwtunnel` directory.

Please refer to the instructions in its `README.md` file.



## Performance implication

This algorithm scales badly when the number of nodes and edges gets larger and larger.

```text
10 nodes      2 edges/n     time: 0.00002 seconds
100 nodes     20 edges/n    time: 0.07790 seconds
100 nodes     40 edges/n    time: 0.38575 seconds
1000 nodes    20 edges/n    time: 1.26458 seconds
1000 nodes    40 edges/n    time: 12.20731 seconds
```

As shown in the the output of the performance testing script `perf_test.py`, when
the system has 1000 nodes and 40 edges per node (40000 edges in total), it takes
over 10 seconds to compute the result.

This indicates that in a real world scale where a tunnel/network system may contain tens of thousands roads or tunnels, in order to ensure the system is responsive (for example, ensure that each query takes at maximum 1 second), we should take the following optimization and scaling approaches:

- use language-level parallelism, such as Python's built-in `multiprocessing` library, which reduces the computing time to roughly $\frac{1}{3} \sim \frac{1}{5}$ on an 8-core server.
- use caching to avoid recomputing the same query, provided that the graph data does not change
- reimplement in a more CPU-efficient language such as C++ (with the help of parallel-programming libraries such as TBB); estimating the speed-up could be 5-10x
- offload the computation to GPU via CUDA graph library or other alternatives
- use infrastructure-level parallelism such as auto-scaling to distribute the query requests onto a cluster

I will cover the infrastructure-level parallelism in the second part.